In [64]:
#Pkg.add("OffsetArrays") # -> Nos permitirá usar matrices que empiecen con índice 0 
using OffsetArrays

# Programación dinámica

Veremos únicamente casos que se calificarían como programación dinámica determinista en horizonte finito (categoría en la que caen la mayoría de problemas de optimización combinatoria determinista).

Utilizaremos dos ejemplos (knapsack y lot sizing) que ya hemos visto y que luego intentaremos combinar/comparar con los métodos expuestos con anterioridad. 

No voy a repasar los fundamentos de la programación dinámica. Si alguien necesita un repaso más allá del nivel introductorio, recomiendo el libro de Denardo porque es uno de los más accesibles y además es barato (E. Denardo, Dynamic Programming: Models and Applications).

# Uncapacitated Lot sizing

Este problema ya lo hemos visto en el diseño de cortes, así que hemos trasladado algunas funciones de la página anterior que nos resultarán útiles


In [84]:
function readULS(path::String)
    f = open(path,"r")
    T = parse(Int64,readline(f)) 
    c=map(x->parse(Int64,x),split(strip(readline(f))))
    h=map(x->parse(Int64,x),split(strip(readline(f))))
    K=map(x->parse(Int64,x),split(strip(readline(f))))
    d=map(x->parse(Int64,x),split(strip(readline(f))))
    close(f)
    T, c, h, K, d
end
function readSimpleInstance()
    T=5
    c=[10 10 10 10 10]
    h=[2 2 2 2 2]
    K=[75 75 75 75 75]
    d=[5 6 7 12 8]
    T,c,h,K,d
end

# T -> número de periodos
# c_t -> costo de comprar una unidad en el periodo t
# h_t -> costo de inventariar una unidad en el periodo t
# K_t -> costo de realizar un pedido en el periodo t
# d_t -> demanda en el periodo t

readSimpleInstance (generic function with 1 method)

In [85]:
#run(`wget -O test.dat https://raw.githubusercontent.com/sbebo/julia-cuts/master/test.dat`)
#run(`cat test.dat`)
T, c, h, K, d = readULS("test.dat")

LoadError: SystemError: opening file test.dat: No such file or directory

## Uncapacitated Lot Sizing. Método 1. Estados asociados al nivel de inventario y recurrencia backward

Éste sería el método tradicional para explicar en clase. Sabemos que en el periodo T+1 el costo es 0 y la última decisión corresponde al periodo T. 

In [98]:
#Pkg.add("OffsetArrays")

function DP_m1(T,c,h,K,d)
    D=sum(d)
    tableCosts = OffsetArray(Int64, 1:T, 0:D)
    tablePath = OffsetArray(Int64, 1:T, 0:D)
    for i=0:d[T]-1
        tableCosts[1,i]=K[T]+c[T]*(d[T]-i) #hacer un pedido y comprar lo que falta hasta d[T]
        tablePath[1,i]=0
    end
    for i=d[T]:D
        tableCosts[1,i]=h[T]*(i-d[T]) #no pedir y pagar inventario
        tablePath[1,i]=i-d[T] 
    end
    for t=2:T #para cada etapa
        period=T-t+1
        for st=0:D
            #hay que diferenciar dos casos
            if st<d[period] #es obligatorio comprar
                #iniciamos con la opción de comprar justo para la semana
                tableCosts[t,st]=K[period]+c[period]*(d[period]-st)+tableCosts[t-1,0]
                tablePath[t,st]=0
                #resto de opciones consisten en comprar desde d[period]-st+1 hasta D-st
                for tr=d[period]-st+1:D-st
                    cost=K[period]+c[period]*tr+h[period]*(st+tr-d[period])+tableCosts[t-1,st+tr-d[period]]
                    if cost < tableCosts[t,st]
                        tableCosts[t,st]=cost
                        tablePath[t,st]=st+tr-d[period]
                    end
                end
            else
                #inicializamos con la opción de no comprar
                tableCosts[t,st]=h[period]*(st-d[period])+tableCosts[t-1,st-d[period]]
                tablePath[t,st]=st-d[period]
                #resto de opciones consisten en comprar desde 1 hasta D-st
                for tr=1:D-st
                    cost=K[period]+c[period]*tr+h[period]*(st+tr-d[period])+tableCosts[t-1,st+tr-d[period]]
                    if cost<tableCosts[t,st]
                        tableCosts[t,st]=cost
                        tablePath[t,st]=st+tr-d[period]
                    end
                end
            end
        end
    end
    ySol=zeros(Int64,T)
    xSol=zeros(Int64,T)
    t=T
    currentState=0
    while t>=1
        if tablePath[t,currentState] > currentState-d[T+1-t]
            ySol[T+1-t]=1
            xSol[T+1-t]=tablePath[t,currentState]+d[T+1-t]
        end
        currentState=tablePath[t,currentState]
        t -= 1
    end
    return ySol,xSol,tableCosts[T,0]
end

DP_m1 (generic function with 1 method)

In [99]:
T,c,h,K,d=readSimpleInstance()
ySol,xSol,totalCost = DP_m1(T,c,h,K,d)
println(ySol)
println(xSol)
println(totalCost)

[1,0,0,1,0]
[18,0,0,20,0]
586


## Uncapacitated Lot Sizing. Método 2. Estados asociados al nivel de inventario y recurrencia forward

Para evitar demasiada similitud con el método anterior, aquí vamos a resolver la recurrencia de una forma diferente a la habitual en el papel (es más es una forma muy usada en implementaciones basadas en la existencia de menos transiciones).

La idea es la siguiente. En el periodo 1 compramos un número de unidades (entre d[1] y D) en el periodo 2 estemos en el estado que estemos, podemos generar todos los descendientes dependiendo (comprando como mínimo d[2]-stock y como máximo sum(d[2:T]) y quedarnos con el mejor entre los generados

Si seguimos así hasta el periodo T, en el periodo T podemos ver cómo llegar al estado final que no tiene inventario.

In [94]:
function DP_m2(T,c,h,K,d)
    tableCosts = OffsetArray(Int64, 2:T, 0:sum(d))
    tablePath = OffsetArray(Int64, 2:T, 0:sum(d))
    #etapa 1
    for tr=d[1]:sum(d)
        tableCosts[2,tr-d[1]]=K[1]+c[1]*tr+h[1]*(tr-d[1])
        tablePath[2,tr-d[1]]=0
    end
    for t=2:T-1
        #empiezo con estado stock=0 para inicializar tableCosts[t+1,:] y tablePath[t+1,:]
        for tr=0:sum(d[t+1:T]) #se inicializa sumando costos hasta ese punto + costo de la compra en ese periodo
            tableCosts[t+1,tr]=tableCosts[t,0]+K[t]+c[t]*(d[t]+tr)+h[t]*tr
            tablePath[t+1,tr]=0
        end
        for s=1:sum(d[t:T]) #estados en los que me puedo encontrar
            for tr=max(0,d[t]-s):max(0,sum(d[t:T])-s) #aqui una transición indica cuántas unidades se van a comprar
                if tr==0
                    cost=tableCosts[t,s]+h[t]*(s+tr-d[t])
                else
                    cost=tableCosts[t,s]+K[t]+c[t]*tr+h[t]*(s+tr-d[t])
                end
                if cost<tableCosts[t+1,s+tr-d[t]]
                    tableCosts[t+1,s+tr-d[t]]=cost
                    tablePath[t+1,s+tr-d[t]]=s
                end
            end
        end
    end
   
    ySol=zeros(Int64,T)
    xSol=zeros(Int64,T)
    totalCost=0
    currentState=0
    #el óptimo corresponde a tener exactamente 0 o d[T] unidades en inventario
    if (tableCosts[T,0]+K[T]+c[T]*d[T]) < tableCosts[T,d[T]] 
        #interesa comprar en el ultimo periodo
        ySol[T]=1
        totalCost=tableCosts[T,0]+K[T]+c[t]*d[T]
        currentState=tablePath[T,0]
    else
        #interesa comprar antes del último periodo
        totalCost=tableCosts[T,d[T]]
        currentState=tablePath[T,d[T]]
    end
    t=T-1
    while t>=2
        if currentState==0
            ySol[t]=1
        end
        currentState=tablePath[t,currentState]
        t -= 1
    end
    ySol[1]=1
    for t=1:T
        if ySol[t]==1
            xSol[t] += d[t]
            for tt=t+1:T
                if ySol[tt]==1
                    break
                end
                xSol[t] += d[tt]
            end
        end
    end
    return ySol,xSol,totalCost
end

DP_m2 (generic function with 1 method)

In [95]:
T,c,h,K,d=readSimpleInstance()
ySol,xSol,totalCost = DP_m2(T,c,h,K,d)
#println("ySol: ",ySol)
#println(xSol)
#println(totalCost)

([1,0,0,1,0],[18,0,0,20,0],586)

## Uncapacitated Lot Sizing. Método 3. Estados asociados al nivel de inventario, recurrencia forward y transiciones limitadas 

Partiendo de las ideas ya vistas en el método 2, uno puede percibir que el conjunto de estados "útiles" es mucho más pequeño que el conjunto de estados "posibles". Por tanto, podríamos aprovechar que la solución es "sparse". 

Para ello vamos a necesitar estructuras de datos algo más complejas y vamos a proceder con la recurrencia forward por comodidad

In [ ]:
function DP_m3(T,c,h,K,d)
    Vector{Vector{Float64}}(0)
    ySol=zeros(Int64,T)
    xSol=zeros(Int64,T)
    totalCost=0
    return ySol,xSol,totalCost
end

## Uncapacitated Lot Sizing. Método 4. Estados asociados a los periodos y recurrencia forward


### Ejercicio Extra

Hay dos cosas en las implementaciones 1 y 2 que son discutibles (sobretodo a nivel de tiempo de cómputo). 

La primera es el uso del paquete "OffsetArrays" que es más lento que una implementación directa. 

La segunda es el usar como primer índice el tiempo y como segundo el estado. Podría ser más rápido usar un orden inverso. 

¿Podría dar una respuesta teórica y validarla empíricamente?